In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

dfb = pd.read_csv('../input/Health_AnimalBites.csv')

In [ ]:
print (dfb.info())

In [ ]:
print (dfb['SpeciesIDDesc'].unique())
print (dfb['BreedIDDesc'].unique())

Testing and trying to understand Chi 2...

So, what about dog bites per year? Is this subject to the Chi 2 test?

In [ ]:
dogs = dfb[dfb['SpeciesIDDesc'] == 'DOG']

In [ ]:
dogs['bite_date'] = pd.to_datetime(dogs['bite_date'], format = '%Y-%m-%d')

There seems to be a problem with the date data. Let´s have a look.

In [ ]:
print (dogs['bite_date'].describe())
print (dogs['bite_date'].isnull().sum())

OK, some cleaning...

In [ ]:
dogs = dogs.sort_values(by='bite_date')

In [ ]:
dogs_cleaned = dogs.loc[:, ['bite_date', 'SpeciesIDDesc', 'BreedIDDesc']]
dogs_cleaned = dogs_cleaned.dropna()

In [ ]:
print (dogs_cleaned)

In [ ]:
dogs_cleaned = dogs_cleaned.reset_index()

In [ ]:
print (dogs_cleaned)

In [ ]:
dogs_cleaned = dogs_cleaned.loc[:3696,]

In [ ]:
dogs_cleaned['bite_date'] = pd.to_datetime(dogs_cleaned['bite_date'], format='%Y-%m-%d')

In [ ]:
print (dogs_cleaned.info())

In [ ]:
dogs_cleaned = dogs_cleaned[dogs_cleaned['bite_date'] >= '2010-01-01']
dogs_cleaned = dogs_cleaned[dogs_cleaned['bite_date'] < '2017-01-01']

In [ ]:
dogs_cleaned['year'] = dogs_cleaned['bite_date'].apply(lambda x: x.year)

In [ ]:
c_t_dogs = pd.crosstab(dogs_cleaned['year'], dogs_cleaned['SpeciesIDDesc'])

In [ ]:
print (c_t_dogs)

I remove 2010 as well, as observations are not relevant

In [ ]:
dogs_cleaned = dogs_cleaned[dogs_cleaned['bite_date'] >= '2011-01-01']
c_t_dogs = pd.crosstab(dogs_cleaned['year'], dogs_cleaned['SpeciesIDDesc'])
print (c_t_dogs)

So, is this suitable for Chi2 test?

In [ ]:
print (stats.chisquare(c_t_dogs['DOG']))

My interpretation of this close-to-zero p-value is that we reject the hypotesis that the distribution of bites per year with breed information is the same as even distribution per year. 

In [ ]:
c_t_breed = pd.crosstab(dogs_cleaned['BreedIDDesc'], dogs_cleaned['SpeciesIDDesc'])
print (c_t_breed)
print (stats.chisquare(c_t_breed['DOG']))

I guess there are too many, too low results. I will reduce the sample to the top quartile.

In [ ]:
print (c_t_breed.describe())

In [ ]:
c_t_breed = c_t_breed[c_t_breed['DOG'] > 24.5]
print (c_t_breed)
print (stats.chisquare(c_t_breed['DOG']))

In [ ]:
c_t_breed = c_t_breed[c_t_breed['DOG'] > 100]
print (c_t_breed)
print (stats.chisquare(c_t_breed['DOG']))

So, I think we reject the hypotesis that even the top biter breeds distribution is the same as expected, even distribution. I guess some dogs bite more than other.